# Doppler Solve

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
from tqdm.auto import tqdm

In [ ]:
def generate(
    nc=1,
    show=True,
    flux_err=1e-6,
    ydeg=15,
    nt=32,
    inc=75,
    veq=50000,
    smoothing=0.075,
    **kwargs
):

    # Instantiate
    map = starry.DopplerMap(ydeg=ydeg, nc=nc, veq=veq, inc=inc, nt=nt)

    # Component surface images
    if nc == 1:
        images = ["spot"]
    elif nc == 2:
        images = ["star", "spot"]
    else:
        raise NotImplementedError("")

    # Component spectra
    mu = np.array([643.1, 642.9])
    sig = 0.025
    dw = map.wav0.reshape(1, -1) - mu.reshape(-1, 1)
    spectra = (1.0 - np.exp(-0.5 * dw ** 2 / sig ** 2))[:nc]

    # Load the component maps
    map.load(images=images, spectra=spectra, smoothing=smoothing)

    # Show
    if show:
        map.show_components(show_spectra=True)

    # Generate unnormalized data
    flux0 = map.flux(normalize=False)
    flux0 += flux_err * np.random.randn(*flux0.shape)

    # Generate normalized data
    flux = map.flux(normalize=True)
    flux += flux_err * np.random.randn(*flux.shape)

    return map, flux0, flux

## Solve for the map

### One component

In [ ]:
settings = dict(flux_err=1e-6, ydeg=15, nt=16, inc=60, veq=50000, smoothing=0.075)
map, flux0, flux = generate(nc=1, **settings)

In [ ]:
# Unnormalized data
map[:, :] = 0
spatial_cov = np.ones(map.Ny)
spatial_cov[1:] = 1e-3
map.solve(
    flux0,
    normalized=False,
    fix_spectrum=True,
    flux_err=settings["flux_err"],
    spatial_cov=spatial_cov,
)
map.show_components(show_spectra=False)

In [ ]:
# Normalized data; tempered solve
map[:, :] = 0
spatial_cov = np.ones(map.Ny)
spatial_cov[1:] = 1e-7
map.solve(
    flux,
    normalized=True,
    fix_spectrum=True,
    flux_err=settings["flux_err"],
    spatial_cov=spatial_cov,
)
map.show_components(show_spectra=False)

### Two components

In [ ]:
# Normalized data; tempered solve + gradient descent
map[:, :] = 0
info = map.solve(
    flux,
    normalized=True,
    fix_spectrum=True,
    flux_err=settings["flux_err"],
    niter=100000,
)
map.show_components(show_spectra=False)